FIXME: 

Change mse (\sigma) to (\sigma**2)

# Overview
In the last lab, we learned how we can use linear regression to estimate how much a voxel responds to a stimulus in an experiment.  We explained the regression problem and derived it's solution. We applied this analysis to fMRI data and estimated regression model weights for each voxel separately.

# Goals
In this lecture, we will study how to make sense of the weights that we estimate from fMRI data. The estimated regression weights of a voxel may be representative of how that voxel responds to stimuli. Alternatively, the presence of noise could have lead to weights that do not correspond to how that voxel responds to the stimuli. To distinguish between these two cases, we need to run a hypothesis test. We will learn to run one hypothesis testing procedure in this lecture..

In [ ]:
# Imports
import neurods as nds
import numpy as np
import os
import matplotlib.pyplot as plt
# Configure defaults for plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.aspect'] = 'auto'
plt.rcParams['image.cmap'] = 'viridis'
%matplotlib inline

A hypothesis test is a statistical test that is used to determine whether there is enough evidence in a sample of data to infer that a certain condition is true in general. A hypothesis test examines two opposing hypotheses: the null hypothesis and the alternative hypothesis.

In the context of brain imaging experiments, we typically have a test for every voxel. For example, for every voxel, we can have the following test:
- null hypothesis: this voxel is not responsive to a given condition (e.g. faces)
- alternative hypothesis: this voxel is responsive to a given condition (e.g. faces)

Another test could be done in contrasting two conditions (e.g. faces vs. objects):
- null hypothesis: this voxel is not responsive to faces more than to places
- alternative hypothesis: this voxel is responsive to faces more than to places

The test can only be defined with a proper specification of its null and alternative hypotheses. Multiple tests could be done with the same data and the estimates we derive from it. A correct formulation of the test will guide the statistical procedure to be done and will help to arrive at a statistically sound hypothesis.

A test typically involves computing a statistic from the data, and then using that statistic either to reject the null hypothesis, or to determine that there is not enough evidence to reject it. We will see here how we can use a t-test, which computes a t-statistic, to perform the second test described above.

- If a test rejects the null-hypothesis, this means that we consider the alternative hypothesis to be true, however, there is some probability $\alpha$ that we may be mistaken.
- If a test fails to reject the null-hypothesis, we cannot say that it is correct, we can only say that there is not enough evidence in favor of the alternative hypothesis.

In a typical fMRI experiment, we convolve the stimulus design matrix with a canonical hemodynamic response function and we use this to estimate weights for each condition at each voxel. We learned in the previous lectures and homework how we can visualize these estimated weights.

In [ ]:
# The ordinary least squares solution for estimating model weights
from numpy.linalg import inv
def OLS(X,Y):
    return np.dot(inv(np.dot(X.T,X)),np.dot(X.T,Y))

# Modeling voxel responses

Remember, we are using regression because we want to model different voxel responses to a set of conditions. 

We do the following steps:
 - Create a design matrix that describes the conditions (or characteristics) of our stimulus time course. (In the lectures we usually provide you with the design matrix. However, when you do an experiment, this is something you have to create in order to draw conclusions. You basically need to specify what the participant observes at each point in time.)
 - Convolve this design matrix with the hemodynamic response function to account for the fMRI BOLD response. 
 - We use linear regression to find out which conditions (e.g. faces, objects, places) are driving a voxel's response. (*Side note: This assumes that each voxel's response was a linear combination of all the conditions (the convolved time courses of the stimuli) and our goal is to recover the parameters (or weights) of this linear combination*)

** Load the design matrix: stimulus time course**

In [ ]:
basedir = os.path.join(nds.io.data_list['fmri'],'categories')
# Load stimulus design matrix
design = np.load(os.path.join(basedir,'experiment_design.npz'))
conditions = design['conditions'].tolist()
print('Conditions: ', conditions)
design_run1 = design['run1']

**Load two runs of fMRI response (run1 and run2) and two runs (run1 and run2) of the stimulus time course:**

When we load two (or more) separate runs of fMRI responses, we normalize (zscore) every run separately. This is an operation that we do in addition to the within voxel normalization we do using the *standardize=True* argument to *nds.fmri.load_data* function. 

Hence, we will use the *zscore* function after we load one run of fMRI data.

In [ ]:
from scipy.stats import zscore
import cortex
sub, xfm = 'S2', 'S2_category_auto'
mask = cortex.db.get_mask(sub, xfm, type='cortical')
fname = os.path.join(basedir, 'S2_categories1_{n}.nii.gz') #S2_categories1_{n}.nii.gz

# Load fmri responses:
Y = np.vstack([zscore(nds.fmri.load_data(fname.format(n=n), mask=mask, standardize=True))
               for n in [1,2]])

# stimuli:
X = np.vstack([design[run] for run in ['run1','run2']])

plt.figure(figsize=(10,4))
plt.imshow(Y)
plt.title('Voxel responses')

plt.figure(figsize=(10,4))
for i, (cond, label) in enumerate(zip(X.T, conditions)):
    plt.plot(cond+i+0.2*i, label=label, lw=2)
plt.title('Condition labels')
_ = plt.legend(frameon=False, bbox_to_anchor=(1.4, 1))

**Convolve the design matrix with the hemodynamic response function to account for the fMRI BOLD response:**

We learned in the previous lectures that we can use the *np.convolve* function to do this.

In [ ]:
from neurods.fmri import hrf as generate_hrf
t_hrf, hrf_1 = generate_hrf(tr=2)
n, d = X.shape

conv_X = np.zeros_like(X)
for i in range(d):
    conv_X[:,i] = np.convolve(X[:,i], hrf_1)[:n]
    
for i, (cond, label) in enumerate(zip(conv_X.T, conditions)):
    plt.plot(cond+i+0.2*i, label=label, lw=2)
    
plt.title('Condition labels')
_ = plt.legend(frameon=False, bbox_to_anchor=(1.4, 1))

**Perform linear regression and estimate model weights:**

We want to find the response of all the voxels in the brain to these 5 different conditions (body, faces, object, places, scrambled). 

Instead of a one dimensional output $Y$, we have a many dimensional output ${\bf Y}$ that describes time and number of voxels.

In [ ]:
print("The size of the fMRI responses: {}".format(Y.shape))
print("The size of the convolved design matrix: {}".format(conv_X.shape))

In [ ]:
weights = OLS(conv_X, Y)
print('The size of the estimated weights is {}'.format(weights.shape))
plt.imshow(weights)
for idx, condition in enumerate(conditions):
    vol = cortex.Volume(weights[idx], sub, xfm, mask = mask)
    __  = cortex.quickflat.make_figure(vol)
    plt.title(condition, fontsize = 30)

- What can we observe from the above plots? Can we draw any conclusions? Where are e.g. different conditions are represented in the brain?

- Can we trust very high weights more than small weights? How can we know what a good threshold is?

# Hypothesis Testing

In order to draw conclusions (or to make inferences) about what conditions are represented in the brain data we first need to estimate an appropriate statistic from the data. We will perform something called a t-test.

### Compute the mean square error

First, we need to compute the **mean squared error**. The weights above were obtained using the matrices conv_X (the convolved design matrix), and Y (the fMRI data matrix). Here are the steps to compute the mean squared error:

#### Breakout Session
- First, use the weights we estimated using linear regression to predict brain responses $\hat {\bf Y}$.
- Second, compute the difference between the recorded brain responses and the predicted brain responses, this is also called, *estimating the error* ${\bf Y-\hat Y}$.
- Then, compute $\boldsymbol \sigma$, the mean squared error: $\frac{1}{N}\sum_{i=0}^{N-1}(Y_i - \hat Y_i)^2$. This will give you a vector corresponding to the mean squared error at every voxel.
- Make a flatmap of $\boldsymbol \sigma$

In [ ]:
# start by using the variables conv_X and weights to estimate Y_hat
# you should estimate a variable called "mse"
# Then use "mse" to plot it in the following way:
# vol = cortex.Volume(mse, sub, xfm, mask = mask)
# __  = cortex.quickflat.make_figure(vol)
# plt.title('mse', fontsize = 30)

### STUDENT ANSWER
Y_hat = np.dot(conv_X, weights)
error = Y - Y_hat
mse = np.mean((Y - Y_hat)**2, axis=0)

vol = cortex.Volume(mse, sub, xfm, mask = mask)
__  = cortex.quickflat.make_figure(vol)
plt.title('mse', fontsize = 30)

### Contrastig conditions

Now we need to define the notion of a contrast. In the last homework, you were asked to subtract the weights for places from the weights for faces. 

This can be called a faces - places contrast. A contrast can be computed using a vector of d numbers. In our case, d=5 because we have 5 conditions.

To compute a **faces - places contrast**, we can use the following **contrast: [0, 1, 0, -1, 0]**. This indicates a contrast weight for each condition. Remember that we have five different conditions in our example (body, faces, object, places, scrambled). So in this contrast, we indicate faces with 1, places with -1 (which will give **faces - places**), and the rest of the conditions with 0.

In [ ]:
c = np.array([0, 1, 0, -1, 0])
print("The size of c: {}".format(c.shape))
print("The size of weights: {}".format(weights.shape))

contrast = np.dot(c, weights)
print("The size of contrast: {}".format(contrast.shape))
vol = cortex.Volume(contrast, sub, xfm, mask = mask)
__  = cortex.quickflat.make_figure(vol)
plt.title('faces - places', fontsize = 30)

What do we use this contrast with? We want to find the regions that are more responsive to faces than to places. Therefore, for every voxel, we have the following:

- null hypothesis: this voxel is not more responsive to faces than to places
- alternative hypothesis: this voxel is more responsive to faces than to places

We want to test whether a particular alternative hypothesis is actually correct, for example, a voxel $v$ does represent faces (in reality, we can never know this fact). 

A t-test is a statistical hypothesis test in which the test statistic follows a Student's t-distribution under the null hypothesis. It can be used to determine if two sets of data are significantly different from each other.

For each voxel $v$, the t-statistic is the contrast value, divided by the standard error of the contrast. The standard error of the contrast is estimated as: $\sqrt{\hat{\sigma_v}^2 c^T (X^T X)^{-1}c}$. Therefore the t-statistic at that voxel is 

$$ t_v = \frac{c^T \hat\beta_v}{\sqrt{\hat{\sigma}^2 c^T (X^T X)^{-1}c}}$$

#### Breakout Session

- Implement a function that takes as input the convolved design matrix, the weights of all voxels, the mean squared error (mse) vector $\boldsymbol \sigma$ and the vector c. This function should output the value of the t-statistic for every voxel. 
- Use this function to estimate the t-statistic for each voxel with the contrast corresponding to faces - places
- Produce a flatmap of those t-statistics 


In [ ]:
# implement the function below, then use it with the existing variables:
# t_statistic = t_stat(conv_X, weights, mse, [0, 1, 0, -1, 0])
# vol = cortex.Volume(t_statistic, sub, xfm, mask = mask)
# __  = cortex.quickflat.make_figure(vol)
# plt.title('t_statistic', fontsize = 30)
from numpy.linalg import inv

def t_stat(X, beta, mse, c):
### STUDENT ANSWER
    num = np.dot(c,beta)
    ctXtXc = np.dot(np.dot(c, inv(np.dot(X.T,X)) ), c)
    denum = np.sqrt(mse* ctXtXc)
    return num/denum

t_statistic = t_stat(conv_X, weights, mse, [0, 1, 0, -1, 0])
vol = cortex.Volume(t_statistic, sub, xfm, mask = mask)
__  = cortex.quickflat.make_figure(vol)
plt.title('t_statistic', fontsize = 30)
    

### Type I and type II error

First, some terminology:

 - Type I error (false positive): The null hypothesis is correct but our test rejects it.
 - Type II error (false negative): The alternative hypothesis is correct and our test does not reject the null hypothesis.

The probability of rejecting a null hypothesis when the alternative is correct, e.g. of correctly concluding that voxel $v$ does represent faces, is called the **power of a test**. The power depends on many properties of an experiment, of the test and of the hypotheses. Critical parameters of the experiment that can affect the power are for example the level of noise in the experiment, the effect size, the number of samples.

We want a test that maximizes power. However, take the following test: always reject the null hypothesis. This test does maximize power, however, it also maximizes type I error. The opposite test: never reject the null hypothesis, minimize type I error, but it also makes the power equal to zero. We therefore want a test that maximizes power while keeping the false positive rate under an acceptable threshold.

We cannot guarantee that we are making no type I errors. However, statistical tests have a guarantee on the probability of error. Depending on which test we use, we estimate a p-value for our observations. A p-value is defined as the probability of obtaining a result equal to or "more extreme" than what was actually observed, when the null hypothesis is true. We then reject the null hypothesis if the p-value is lower than some threshold. 0.05 is for example a common threshold for tests. If we reject with this rate, it means that the probability of us making a mistake is less than 0.05. 

Let's say the null hypothesis is actually true. Using a threshold of 0.05 actually means that if we could somehow repeat the experiment a large number of times, we would (falsely) reject the null hypothesis 5% of those times.


### p-value computation

When using a t-test, we consider the t-statistic to have a specific t-distribution under the null hypothesis. This distribution is determined by it's **degree of freedom $\nu$**. For us, $\nu$ corresponds to the number of data points used to estimate the statistic, i.e. the original number of samples (240).

What we do in a test is to estimate a p-value for the observed statistic. A p-value is the probability of observing the statistic we obtain, given that the null hypothesisis correct.

To obtain the p-value of a t-statistic, we compute the probability of seeing a t-statistic this extreme under the t-distribution with degree of freedom $\nu$. If this p-value is very small, it means that this t-statistic is surprising under the null distribution. This suggests that maybe the alternative hypothesis is correct.

In [ ]:
nu = Y.shape[0]
from scipy.stats import t as tdistribution
x = np.linspace(-10, 10, num=10001)

# Compute the probability density function
t_pdf = tdistribution.pdf(x, nu)
plt.figure()
plt.plot(x, t_pdf)
plt.xlabel('x')
plt.ylabel('P(X)')

# Compute the cummulative density function
t_cdf = tdistribution.cdf(x, nu)
plt.figure()
plt.plot(x, t_cdf)
plt.xlabel('x')
plt.ylabel('P(X<=x)');

We can use the function *tdistribution.cdf* to estimate a p-value for the contrast at every voxel. 

cdf(x) is the probability of obtaining a value smaller than x. We want the p-value, which is the probability of obtaining a value higher than x, which is 1-cdf(x). 

In [ ]:
p_statistic = 1 - tdistribution.cdf(t_statistic,nu)
vol = cortex.Volume(p_statistic, sub, xfm, mask = mask)
__  = cortex.quickflat.make_figure(vol)
plt.title('p_statistic', fontsize = 30)

vol = cortex.Volume(-np.log10(p_statistic), sub, xfm, mask = mask)
__  = cortex.quickflat.make_figure(vol)
plt.title('p_statistic, log scale', fontsize = 30)

### Significance threshold
In order to run the test we need to state a significance threshold $\boldsymbol\alpha$. After computing the p-values, we can use this significance threshold to decide whether we will reject the null hypothesis. This means we reject it when $p<=\alpha$.

- What happens if we chose a significance threshold $\alpha=0.05$? 

In [ ]:
vol = cortex.Volume((p_statistic<=0.05)*1., 
                    sub, xfm, mask=mask, vmin=-2, vmax=2)
__  = cortex.quickflat.make_figure(vol)
plt.title('p<=0.05', fontsize = 30);

## Multiple comparison correction

Check out the blog post: https://blogs.scientificamerican.com/scicurious-brain/ignobel-prize-in-neuroscience-the-dead-salmon-study/

This is about the following paper: 
Bennett et al. "Neural Correlates of Interspecies Perspective Taking in the Post-Mortem Atlantic Salmon: An Argument For Proper Multiple Comparisons Correction" Journal of Serendipitous and Unexpected Results, 2010.

Bennett et al. were able to show that a group of voxels in a dead salmon were implicated in processing social cues from images. This study was designed to show the perils of failing to correct for multiple comparisons.

Let's say there is a game in which a person has to pick a slot out of 100 and win a big prize. On the first try, that person guesses the correct slot.
- What is the probability of that event happening by chance?
- Would you think this person was previously told about which slot was correct?

Now assume you have 10000 people playing this game. On their first try, 109 people guess the correct slot and win the prize.
- Would you be think these people were previously told about which slot was correct?
- Would your assumption change if 5000 people guess the correct slot on their first try?
- How does this relate to our problem?

Multiple methods exist to perform multiple comparison correction. Some methods control the
**Familywise Error Rate (FWER)**, which is the chance of getting any false positives. 

One such method is the **Bonferroni correction**. When correcting for $M$ multiple comparisons, the Bonferroni correction consist of using a rate of $\alpha/M$ instead of $\alpha$.

This is a conservative test: to avoid false positives, it makes the rate very small and thus might fail to reject many true positives.

In [ ]:
alpha = 0.05
M = Y.shape[1]
alpha_bonferroni = alpha/M
vol = cortex.Volume( (p_statistic<=alpha_bonferroni) * 1.0 , 
                    sub, xfm, mask = mask, vmin = -2, vmax = 2)
__  = cortex.quickflat.make_figure(vol)
plt.title('Bonferroni correction', fontsize = 30)

Other, less conservative methods that control the FWER exist. We will not go over them here.

Another way to correct for multiple comparisons is to control the **False Discovery Rate (FDR)**, i.e. limit the proportion of false positives among the rejected tests. One famous method is the Benjamini Hochberg Procedure. We implement it for you below. We can use it to control the FDR at q=0.05:

In [ ]:
def FDR_BH(p_vals, q):
    # sort the p_values
    p_val_sorted = np.sort(p_vals)
    M = p_val_sorted.shape[0]
    threshold = 0
    # go the p_values in order and find the largest k where P(k) <= k/M*q
    for idx,p in enumerate(p_val_sorted):
        if p <= idx*q/M:
            threshold = p
        else:
            break
    return threshold

threshold = FDR_BH(p_statistic, q=0.05)
vol = cortex.Volume((p_statistic<=threshold)*1.0, sub, xfm, mask=mask, 
                    vmin = -2, vmax = 2)
__  = cortex.quickflat.make_figure(vol)
plt.title('FDR correction', fontsize = 30);

You can see how this method rejects the null hypothesis at more voxels. Both methods control for multiple comparisons.

Other methods exist that use spatial contraints: for example, only contiguous voxels forming a cluster of a certain size are selected. The size of the cluster is determined by the test.

## Predicting withheld data:

Another method for testing if the weights of a learned model are meaningful and not due only to chance is to test them to predict new, unseen data. The idea is that if the weights we estimated are indicative of how the brain responds to the experimental conditions, then we can use them to predict the brain response for new data. Here, we introduce concepts that are very important for the statistics and machine learning fields:

- Training set: is the part of the data you use to estimate your model. You can use this data as you wish. We will discuss overfitting next week and see why you might want to be careful with how much of the variance of this data you want your model to predict.
- Test set: this test should remain untouched until the very end of your analysis, where you only use it to report your results. You should never go back to your analysis and change any parameters based on the performance of your model on the test set. 

We did not use yet the third run of our experiment. We will load it here and use it to test the performance of our model on it:

In [ ]:
import cortex
sub, xfm = 'S2', 'S2_category_auto'
mask = cortex.db.get_mask(sub, xfm, type='cortical')
fname = os.path.join(basedir, 'S2_categories1_{n}.nii.gz') #S2_categories1_{n}.nii.gz
# fmri responses:
Y_test = np.vstack([zscore(nds.fmri.load_data(fname.format(n=n), mask=mask, standardize=True)) for n in [3]])
# stimuli:
X_test = np.vstack([design[run] for run in ['run3']])
n_test = X_test.shape[0]

plt.figure(figsize=(10,4))
for i, (cond, label) in enumerate(zip(X_test.T, conditions)):
    plt.plot(cond+i+0.2*i, label=label, lw=2)
plt.title('Condition labels')
_ = plt.legend(frameon=False, bbox_to_anchor=(1.4, 1))

conv_X_test = np.zeros_like(X_test)
for i in range(d):
    conv_X_test[:,i] = np.convolve(X_test[:,i], hrf_1)[:n_test]
    

#### Breakout Session

Using the weights you have estimated before:
- First, use conv_X_test to predict the activity $ {\bf \hat Y_{test}}$.
- Second, compute the difference between the recorded brain responses and the predicted brain responses, this is also called, *estimating the error* ${\bf Y_{test}-\hat Y_{test}}$.
- Then, compute $\bf \boldsymbol \sigma_{test}$, the mean squared error $\frac{1}{N}\sum_{i=0}^{N-1}({Y_{test}}_i -  {\hat Y_{test}}_i)^2$. This will give you a vector corresponding to the mean squared error at every voxel.
- Compute the coefficient of determination, which estimates how much of the data is being predicted:
    $R^2_{\bf test} = 1 - \frac{\bf \boldsymbol \sigma_{test}}{var({\bf Y_{test}})} $.
    Since we have already normalized every voxel to have a variance of 1, you can simplity the computation to:
    $R^2_{\bf test} = 1 - \bf \boldsymbol \sigma_{test} $
- Produce a flatmap of $R^2_{\bf test}$
- Then, use the previously computed $\boldsymbol \sigma$ using training data to produce a flatmap of $R^2_{\bf train} = 1 - \bf \boldsymbol \sigma_{} $. Use the same scale for both (see below)


In [ ]:
# start by using the variables conv_X_test and weights to estimate Y_hat_test
# you should estimate a variable mse_test
# then use to plot it the following:
# vol = cortex.Volume(1-mse_test, sub, xfm, mask = mask, vmin = 0, vmax = 0.5)
# __  = cortex.quickflat.make_figure(vol)
# plt.title('R2 test', fontsize = 30)
# then plot the training R2:
# vol = cortex.Volume(1-mse, sub, xfm, mask = mask, vmin = 0, vmax = 0.5)
# __  = cortex.quickflat.make_figure(vol)
# plt.title('R2 train', fontsize = 30)

### STUDENT ANSWER
Y_hat_test = np.dot(conv_X_test,weights)
error = Y_test - Y_hat_test
mse_test = np.mean((Y_test - Y_hat_test)**2, axis=0)
vol = cortex.Volume(1-mse_test, sub, xfm, mask = mask, vmin = 0, vmax = 0.5)
__  = cortex.quickflat.make_figure(vol)
plt.title('R2 test', fontsize = 30)
vol = cortex.Volume(1-mse, sub, xfm, mask = mask, vmin = 0, vmax = 0.5)
__  = cortex.quickflat.make_figure(vol)
plt.title('R2 train', fontsize = 30);

- What is the difference between the two plots?

Additionally, one of the big fallacies in the fMRI litterature was something called double dipping. you should be careful to never use the same data to make and test your hypotheses! 